In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from io import StringIO
import pprint
import pandas_profiling
import re
import random
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import display, clear_output, HTML

import math
from collections import Counter
import heatmap
import gc
from heatmap import heatmap, corrplot
from ipywidgets import Layout, Button, Box
import plotly.express as px
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from plotly.graph_objs import graph_objs as go 
import plotly.offline
from plotly.offline import iplot
import threading
import time
import ipyfilechooser
from ipyfilechooser import FileChooser
#$ pip install ipyupload
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
from ipyupload import FileUpload
from itertools import cycle, chain, combinations
import dill
from IPython.core.display import display, HTML
#display(HTML("<style>.container {width:80% !important;}</style>"))
%matplotlib inline
import shap

from aequitas.group import Group
from aequitas.bias import Bias
from aequitas.fairness import Fairness
from aequitas.plotting import Plot
from aequitas.preprocessing import preprocess_input_df

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
class helper_methods():

    def __init__(self):
        self.text_color = "green"
        self.progressBar = widgets.FloatProgress(
            value=0.0, 
            min=0.0, 
            max=100.0,
            #layout = auto_width_layout,
        )
        
        self._percent_slider = "tbd"               
        self._top_x_slider  = "tbd"
        
    
    def display_html(self, text, color, size):
        content = "<" + size + ">"  + "<text style='color:" + color + "'>" + text + "</text>" + "</" + size + ">" 
        display(HTML(content))
        
      
    def work_in_progress(self):
        for i in range(100):
            if self.progressBar.value != 100:
                self.progressBar.value = i# float(i+1)/total
                time.sleep(0.3)
            if self.progressBar.value == 100:
                break
            
            

                
    #################################################################################################
    #  
    # 
    #################################################################################################
    def view_correlations(self, df, column_list, column_list_with_Y, protected_column_list, y_col ):
       
        feature_set_with_y = df
        out1 = widgets.Output(layout={})
        out2 = widgets.Output(layout={})
        out3 = widgets.Output(layout={})
        out4 = widgets.Output(layout={})
        tab_contents = [out1, out2, out3]
        
        children = tab_contents
        tab = widgets.Tab()
        tab.children = children
        
        tab.set_title(0, "Correlation plot")
        tab.set_title(1, "Pair Plot")
        tab.set_title(2, "Feature distribution")
      
        display(out4)
        display(self.progressBar)
        display(tab)
        
        column_to_count = []
        column_to_corr = []
        for  col in column_list_with_Y:
            if len(feature_set_with_y[col].unique ())< 10:
                column_to_count.append(col)
            else:
                column_to_corr.append(col)
        
        def f_cor(x ): #local method within view_correlations() funct
                self.progressBar.value = 0
                thread = threading.Thread(target=self.work_in_progress)
                thread.start()
                plt.clf()
                plt.figure(figsize=(12, 12))     
                fig1 = corrplot(feature_set_with_y[column_list_with_Y].corr(), size_scale=x, marker="o");       
                plt.show(fig1)
                plt.clf()
                plt.close(fig1)
                self.progressBar.value = 100
                        
        
        def f_cor_val( a,b ): #local method within view_correlations() funct
                self.display_html ("The correlation value is " + str(feature_set_with_y[a].corr(feature_set_with_y[b])), self.text_color, "p")
        
        with out1:
            interact(f_cor, x = widgets.IntSlider(description = "slide to scale", min=0, max=70000, step=100, value=500, continuous_update=False), );
        
            interact(f_cor_val, 
                        a = widgets.Dropdown(description = "Column 1", options = column_list_with_Y),
                        b = widgets.Dropdown(description = "Column 2", options = column_list_with_Y)
                );
        
                    
        def f_pp(_hue): #local method within view_correlations() funct
            self.progressBar.value = 0
            thread = threading.Thread(target=self.work_in_progress)
            thread.start()
            with out4:
                clear_output(wait = False)
                self.display_html("Calculating pair plot..", self.text_color, "p")
            
   
            plt.figure(figsize=(6, 6))      
            pp = sns.pairplot(feature_set_with_y[column_to_corr+[_hue]], corner=True, hue=_hue, diag_kind = 'kde', height=3, plot_kws = { 'alpha': 0.6,'s': 80, 'edgecolor': 'k'});  
            pp._legend.get_title().set_fontsize(20)   
            plt.show()
            plt.clf()
            plt.close()
            self.progressBar.value = 100
                
            with out3: 
                clear_output(wait = False)
                self.progressBar.value = 100
                a = 0
                b = 0
                    
                num_plots = len(column_to_count)
                num_rows = math.ceil(num_plots/2)  
                f, axes = plt.subplots(num_rows, 2, figsize=(15, num_rows*5))
                for  col in column_to_count:
                    sns.countplot(x = col, data = feature_set_with_y, ax=axes[a, b], hue=_hue)
                    a = a + 1
                    if a == (num_rows):
                        b = b + 1
                        a = 0
                plt.show()
                plt.clf()
                plt.close()
            
            with out4:
                clear_output(wait = False)
                self.display_html("Plots complete! Generating pandas profiling.", self.text_color, "p")
                self.progressBar.value = 100
            
       
        with out2:
            clear_output(wait = False)
            interact(f_pp, _hue = widgets.Dropdown(description = "Hue based on", 
                                                           options = protected_column_list));
            
            

            
            
    #################################################################################################
    #  VIEW STATISTICS AROUND. THE PROTECTED FEATURES/ATTRIBUTES
    # 
    #################################################################################################
            
    def view_protected(self, protected_attributes_list, 
                             y_value, data_frame, 
                            _w=600, _h=600, 
                            y_high_positive = True,
                            persist_impact_col = False,
                            output_type_binary = False,
                            show_outcome_info = True):
        protected = []
        protectedCount = []
        protectedPercentage = []
        HIGH_RANGE_POSITIVE = y_high_positive
        
        out1 = widgets.Output(layout={})
        out2 = widgets.Output(layout={})
        out3 = widgets.Output(layout={})
        out4 = widgets.Output(layout={})
        
        if show_outcome_info == True:
            tab_contents = [out1, out2, out3]
        else:
            tab_contents = [out1, out2]
        
        children = tab_contents
        tab = widgets.Tab(style={'description_layout':'auto', 'title_layout':'auto'})
        tab.children = children
        
        tab.set_title(0, "Group Representation in data")
        tab.set_title(1, "Outcome distribution")
        tab.set_title(2, "Group Positive outcome representation")
        
        local_layout = {'width': 'auto', 'visibility':'visible'}
        local_layout_hidden  = {'width': 'auto', 'visibility':'hidden'}
        local_style = {'description_width':'initial'}
        display(out4)
        display(tab)
        try:
            colType = data_frame[y_value].dtype
            if (colType != str and colType != object):
                ####### view distribution of groups in the data
                with out1:
                    clear_output(wait = False)
                    fig = self.plot_donut(protected_attributes_list,
                               data_frame,
                               w=_w, h=_h,
                               y_high_positive = HIGH_RANGE_POSITIVE,
                               title = "Representation of Protected group(s) in the data",
                               show_individual = False);
                    fig.show()
                    
                    def show_donut_for(choose): #local method within view_protected() funct
                        #plot the representation of data in the dataframe per protected group
                        with out4:
                            clear_output(wait = False)
                            self.display_html("loading group representation in data...", self.text_color, "p") 
                        fig = self.plot_donut([choose],
                               data_frame,
                               w=_w, h=_h,
                               y_high_positive = HIGH_RANGE_POSITIVE,
                               title = "Representation of " + choose + " in the data",
                               show_individual = False);
                        fig.show()
                        del fig
                        with out4:
                            clear_output(wait = False)
                            self.display_html("Finished loading", self.text_color, "p")
                    if len (protected_attributes_list) > 1:
                        interact(show_donut_for, 
                            choose = widgets.Dropdown(description = "Select attribute", 
                                                      options = protected_attributes_list,
                                                     layout = local_layout,
                                                     style = local_style),
                                );
                
                
                ############## view distribution of results across groups
                with out2:
                    clear_output(wait = False)
                        #plot the distribution of Y-value per protected group
                    def show_distribution(choose, curve_type): #local method within view_protected() funct
                        #plot the representation of data in the dataframe per protected group
                        with out4:
                            clear_output(wait = False)
                            self.display_html("loading outcome distribution...", self.text_color, "p")
                        if choose != "--select--":
                            self.plot_distribution( [choose],
                                       y_value, 
                                       data_frame, 
                                       w=_w, h=_h, 
                                       y_high_positive = HIGH_RANGE_POSITIVE,
                                       curve_type = curve_type);
                        with out4:
                            clear_output(wait = False)
                            self.display_html("Finished loading", self.text_color, "p")
                    
                    def show_distribution_count(choose): #local method within view_protected() funct
                        if choose != "--select--":
                            output_values = data_frame[y_value].unique()
                            group_types = data_frame[choose].unique()
                            fig = go.Figure()
                            for val in output_values:
                                temp = data_frame[data_frame[y_value]==val]
                                fig.add_trace(go.Histogram(
                                                        x=temp[choose],
                                                        y=temp[y_value],
                                                        name= "Output " + str(val),
                                                        histfunc="count",
                                                        opacity=0.75))
                                fig.update_layout(
                                            title_text='Outcome distribution', # title of plot
                                            xaxis_title_text=choose, # xaxis label
                                            yaxis_title_text='Count', # yaxis label
                                            bargap=0.2, # gap between bars of adjacent location coordinates
                                            bargroupgap=0.1, # gap between bars of the same location coordinates
                                            #legend_title_text=y_value+" output"
                                            autosize=False
                                            )

                        fig.show()

                    if output_type_binary == False:
                        interact(show_distribution, 
                                 choose = widgets.Dropdown(description = "Select attribute",
                                                           options = ["--select--"] + protected_attributes_list,
                                                           layout = local_layout,
                                                           style = local_style),
                                 curve_type = widgets.Dropdown(description = "Select Curve type",
                                                               options = {"Kernel Density Estimation":"kde","Normal Distribution":"normal"},
                                                               layout = local_layout,
                                                               style = local_style),
                                );
                    
                    elif output_type_binary == True:
                        interact(show_distribution_count, 
                                 choose = widgets.Dropdown(description = "Select attribute",
                                                           options = protected_attributes_list,
                                                           layout = local_layout,
                                                           style = local_style),
                                );
                        

                with out4:#plot the representation of data in the dataframe per protected group
                    clear_output(wait = False)
                    self.display_html("loading outcome representation per group...", self.text_color, "p")      
                ############## view distribution of positive outcome across groups
                
                with out3:
                    clear_output(wait = False)
                    if HIGH_RANGE_POSITIVE == True:
                        impactTxt = "<b>Positive</b>"
                    if HIGH_RANGE_POSITIVE == False:
                        impactTxt = "<b>Negative</b>"
                    x = "The Impact of a high output(ranking) on an individual or group is <b>" + impactTxt + "</b>"
                    self.display_html(str(x), self.text_color, "p")
                    self.display_html("Select the decision boundary between a positive and negative outcome for the purpose of the report.", self.text_color, "p")

                    def show_outcome(choose, percentile_slider, top_x_slider): #local method within view_protected() funct
                        #plot the representation of data in the dataframe per protected group
                        self._percent_slider.layout = local_layout_hidden
                        self._top_x_slider.layout = local_layout_hidden
                        with out4:
                            clear_output(wait = False)
                            self.display_html("loading outcomes across groups...", self.text_color, "p")
                        if choose == "Mean":
                            self._percent_slider.layout = local_layout_hidden
                            self._top_x_slider.layout = local_layout_hidden
                             #plot the distribution of above and below average score(y-value) per protected group
                            self.plot_outcome_representation(protected_attributes_list,
                                                             y_value, data_frame,
                                                             w=_w, h=_h,
                                                             y_high_positive = HIGH_RANGE_POSITIVE,
                                                             percentile = None,
                                                             top_x = None,
                                                             persist_impact_col = persist_impact_col)
                                 
                        if choose == "Percentile":
                            self._percent_slider.layout = local_layout
                            self._top_x_slider.layout = local_layout_hidden
                        #plot the distribution of above and below average score(y-value) per protected group
                            self.plot_outcome_representation(protected_attributes_list,
                                                             y_value, data_frame,
                                                             w=_w, h=_h,
                                                             y_high_positive = HIGH_RANGE_POSITIVE,
                                                             percentile = percentile_slider,
                                                             top_x = None,
                                                             persist_impact_col = persist_impact_col)
                        if choose == "Top-n":
                            self._percent_slider.layout = local_layout_hidden
                            self._top_x_slider.layout = local_layout
                        #plot the distribution of above and below average score(y-value) per protected group
                            self.plot_outcome_representation(protected_attributes_list,
                                                             y_value, data_frame,
                                                             w=_w, h=_h,
                                                             y_high_positive = HIGH_RANGE_POSITIVE,
                                                             percentile = None,
                                                             top_x = top_x_slider,
                                                            persist_impact_col = persist_impact_col)

                    # see https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html
                        with out4:
                            clear_output(wait = False)
                            self.display_html("Finished loading!", self.text_color, "p")
                    
                    
                    if output_type_binary == True:
                        _choose = widgets.Dropdown(
                                description = "Positive v's Negative determined by", 
                                options = ["Mean"],
                                layout = local_layout,
                                style = local_style)
                    else:
                        _choose = widgets.Dropdown(
                                description = "Positive v's Negative determined by", 
                                options = ["Mean","Top-n","Percentile"],
                                layout = local_layout,
                                style = local_style)
                    
                    self._percent_slider = widgets.IntSlider(
                                description = "Select Percentile", 
                                min=0, max=100,
                                step=1, value=80, 
                                continuous_update=False,
                                layout = local_layout_hidden,
                                style = local_style)
                    
                    self._top_x_slider = widgets.IntSlider(
                                description = "Select n for Top_n", 
                                min=10, max=1000,
                                step=10, value=100, 
                                continuous_update=False,
                                layout = local_layout_hidden,
                                style = local_style) 
                    
                    if show_outcome_info == True:
                        interact(show_outcome, 
                                 choose = _choose,
                                 percentile_slider = self._percent_slider, 
                                 top_x_slider = self._top_x_slider,);
                    
                    
                    
                    with out4:#plot the representation of data in the dataframe per protected group
                        clear_output(wait = False)
                        self.display_html("Complete!", self.text_color, "p")
                
                if (colType == str or colType == object):
                    self.display_html("Output label is not number and should be converted", self.text_color, "h4")
        except Exception as e:
            self.display_html("Something went wrong in view_protected method", self.text_color, "h4")
            print (e)
#         gc.collect()
        with out4:#plot the representation of data in the dataframe per protected group
            clear_output(wait = False)
            self.display_html("finished loading", self.text_color, "p")
        
          
    #################################################################################################
    #  
    # 
    #################################################################################################         
    def create_label (self, row):#method local to this method
            
            names = list (row.index)
            values = list( row.values)
            text = ""
            for i in range (len(names)):
                text = text + ":" + names[i] + "_" + str(values[i])
            text = text[1:]            
            return text       
    #################################################################################################
    #  
    # 
    #################################################################################################       
    def plot_donut(self, attributes_list, data_frame, w=800, h=800, y_high_positive = True, title = "Result",
                         show_individual = True):
    
        num_of_donuts = len(attributes_list)
        if num_of_donuts > 6:
            num_of_donuts = 6
            display (HTML("showing only the first 6 attributes"))
        
        sequential_color_list = [
            px.colors.sequential.Blues,
            px.colors.sequential.Greens, 
            px.colors.sequential.Oranges, 
            px.colors.sequential.Purples,
            px.colors.sequential.Reds,
            px.colors.sequential.Greys,
            px.colors.sequential.algae,
            px.colors.sequential.amp]
    
        color_pool = cycle(sequential_color_list)
    
        pie_list = []    
        a_l = []
        labels_arr = []
        values_arr = []
        color_arr = []
        annotations_arr = []
        annotate = dict(text='woops', 
            x=0.5, y=0.6, 
            font_size=15, 
            showarrow=False)
                     
        def create_label (row):#method local to this method
            
            names = list (row.index)
            values = list( row.values)
            text = ""
            for i in range (len(names)):
                text = text + ":" + names[i] + "_" + str(values[i])
            text = text[1:]            
            return text
            
        for a, pos in zip (attributes_list, range(len(attributes_list))):
            a_l.append(a)
            annotate['text'] = a
            annotate['y'] = annotate['y']-0.05
            annotations_arr.append(annotate.copy())
            #df = data_frame[a_l+[y]].groupby(a_l).count().reset_index().rename(columns={y: "values"})
            data_frame["count"] = 0
            df = data_frame[a_l+["count"]].groupby(a_l).count().reset_index().rename(columns={"count": "values"})
            df['labels'] =  df.apply(lambda row : self.create_label(row[a_l]), axis = 1) 
            #df['labels'] = df[a_l].astype(str).sum(axis=1) 
            df['values'].fillna(0,inplace=True)
            c = []
            s = []
            if pos == 0:
                for l in range (len(df['labels'].values)):
                    c.append(next(color_pool)[0])
                    if l >= len(sequential_color_list):
                        l = l - len(sequential_color_list)
                    s.append(l)
                df['colors'] = c
                df['color_pool_pos'] = s
                
            else:
                temp_list = list(df['values'].values)
                for count, color_index in zip(prev_counts, prev_color_pool) :
                    match = 0
                    for value, pos in zip (temp_list, range(len(temp_list))):
                        s.append(color_index)
                        try:
                            c.append (sequential_color_list[color_index][pos+1])
                        except:
                            c.append (sequential_color_list[color_index][2])
                        match = match + value
                        if match == count:
                            del temp_list[0:pos+1]
                            break
                df['colors'] = c
                df['color_pool_pos'] = s
            labels_arr.append (df['labels'])
            values_arr.append (df['values'])
            color_arr.append (df['colors'])
        
            prev_counts = df['values'].values
            prev_color_pool = df['color_pool_pos'].values

        
        hole = 0.8
        x1 = 0
        x2 =1
        y1 = 0
        y2 = 1
        adjust = round((1.0 - hole)* 0.5,2) 
        for x in range (num_of_donuts):
            pie_list.append(go.Pie(
            hole=hole, #Sets the fraction of the radius to cut out of the pie. Use this to make a donut chart
            sort=False,
            direction='clockwise',
            domain={'x': [x1, x2], 'y': [y1, y2]},
            values=values_arr[x],
            labels=labels_arr[x],
            #text = my_text,
            textinfo='label+percent',
            textposition='inside',
            name=attributes_list[x],
            marker={'colors': color_arr[x],'line': {'color': 'black', 'width': 1}}
            ))
            hole= round(hole - adjust, 2)
            x1 = round (x1 + adjust, 2)
            x2 = round (x2 - adjust, 2)
            y1 = round (y1 + adjust, 2)
            y2 = round (y2 - adjust, 2)
         
    
        fig = go.FigureWidget(data=pie_list)#need to reverse the order?
        fig.update_layout(autosize=False,
                          width=w,
                          height=h,
                          margin=dict(l=50,r=50,b=100, t=100,pad=4),
                          title=str(a_l),
                          #Add annotations in the center of the donut pies.
                          annotations=annotations_arr,
                          legend_orientation="h",
                           #paper_bgcolor='rgba(113, 136, 136, 1)', #for transparent set to (0,0,0,0)
                          #plot_bgcolor='rgba(113, 136, 136, 1)',
                    )

        fig.update_traces(textposition='inside')
        fig.update(layout_title_text=title,
               layout_showlegend=True )  
        
        
        
         # # # # # Now create one donut per protected attribute for a clearer view if the call specifies this# # # # # # 
        if show_individual == True:
            specs = []
            fig2 = ""
            full_title =  ''
            if len(attributes_list) > 1:
                for x in range(len(attributes_list)):
                    specs.append ({'type':'domain'})
                fig2 = make_subplots(rows=1, cols=len(attributes_list), specs=[specs])
                for attribute, index in zip (attributes_list, range(len(attributes_list))):
                    full_title =  full_title + "/ " + attribute
                    df = data_frame[[attribute]+["count"]].groupby([attribute]).count().reset_index().rename(columns={"count": "values"})
                    pie = go.Pie(
                    hole=hole, #Sets the fraction of the radius to cut out of the pie. Use this to make a donut chart
                    sort=False,
                    direction='clockwise',
                    values=df["values"],
                    labels=df[attribute],
                    textinfo='label+percent',
                    textposition='inside',
                    name=attribute,
                    title = attribute,
                    #insidetextfont=dict(family="sans serif",size=18,color="crimson"),
            
                    marker={'colors': color_arr[x],'line': {'color': 'black', 'width': 1}} )
                    fig2.add_trace(pie, 1, index+1)
    
                fig2.update_layout(autosize=False,
                          width=600,
                          height=300,
                         #margin=dict(l=50,r=50,b=100, t=100,pad=4),
                  
                          )
    
    
                fig2.update_traces(textposition='inside')
                fig2.update(layout_title_text='Representation per attribute', layout_showlegend=True)   

                fig2.show()
        #as this can be a pointer to the input, clean it up
        data_frame.drop(["count"], axis=1, inplace = True)
        gc.collect()
        return fig #should also return fig2
        
        
        
        
    #################################################################################################
    #  
    # METHOD USED TO PLOT THE DISTRIBUTION OF THE OUTCOME ACROSS GROUPS
    #################################################################################################
    
    def plot_distribution(self, attributes_list, y, data_frame, w=800, h=800, y_high_positive = True, curve_type = "kde"):
        try:
            dist_arr = []
            g_labels = []
            for attribute, pos in zip (attributes_list, range(len(attributes_list))):
                groups = data_frame[attribute].unique()
                for value in range(len(groups)):
                    df = data_frame[ data_frame[attribute] == groups[value]]
                    dist_arr.append(df[y])
                    g_labels.append(attribute + "-" + str(groups[value]))
                # Add histogram data
                # Group data together
                hist_data = dist_arr
                group_labels = g_labels
                #Add the dist of all combined groups
                hist_data.append(data_frame[y])
                group_labels.append("All")
            
                # Create distplot with custom bin_size
                fig = ff.create_distplot(hist_data, group_labels, curve_type = curve_type, show_hist=True) #, bin_size=[.1, .25, .5, 1]
                # Add title
                fig.update_layout(
                    autosize=False,
                    width=900,
                    height=500,
                    margin=dict(l=50,r=50,b=100, t=100,pad=4),
                    paper_bgcolor="LightSteelBlue",
                    title=y +' distribution across ' + attribute,
                    xaxis=dict(range=[data_frame[y].min(), data_frame[y].max()])
                )

                f2 = go.FigureWidget(fig)
                f2.show()
                #f = fig.show(renderer="notebook")
                groups = []
                dist_arr = []
                g_labels = []

        except Exception as e:
            self.display_html("Something went wrong generating the distribution, change the distribution type and ensure group is represented sufficiently to generate dist", 
                              self.text_color, "h4")
            print (e)
        gc.collect()
        
    
    #################################################################################################
    #  
    # 
    #################################################################################################
    def plot_outcome_representation(self, attributes_list, 
                                    y_value, data_frame, 
                                    w=800, h=800, 
                                    y_high_positive = True,
                                    percentile = None,
                                    top_x = None,
                                    persist_impact_col = False):
        
        
        num_of_attributes = len(attributes_list)
        if num_of_attributes > 6:
            num_of_attributes = 6
            display (HTML("showing only the first 6 attributes"))
        
        #create a new column that categorises positive outcome by default, as above average score and negative 
        #outcome as below average score
        y = y_value #Sort the naming out here later
        text = ""
        HIGH_RANGE_POSITIVE = y_high_positive
        yMin = round(data_frame[y_value].min(), 10)
        yDivPoint = data_frame[y_value].mean()
        yMax = data_frame[y_value].max()   
        
        outcome = ['', '']      
        if HIGH_RANGE_POSITIVE == True:
            # min to mean is neg, mean to max is pos
            outcome = ["Neg", "Pos"]
            #Rather than having defined bins of [yMin, yMean, yMax] base the display on precentiles.
            if not percentile == None:
                yDivPoint = data_frame[data_frame[y_value] >= np.percentile(data_frame[y_value],percentile)].min()[y_value]
            if not top_x == None:
                #Ascending means smallest to largest,
                _text = "Top n = "+ str (data_frame.sort_values(y_value,ascending = True).head(top_x).shape[0])
                self.display_html(_text, self.text_color, "p")
                yDivPoint = data_frame.sort_values(y_value,ascending = False).head(top_x).min()[y_value]
                    
        if HIGH_RANGE_POSITIVE == False:
            outcome = ["Pos", "Neg"]
            # min to mean is pos, mean to max is neg
            #Rather than having defined bins of [yMin, yMean, yMax] base the display on precentiles.
            if not percentile == None:
                _text = "bottom percentile = " + str(percentile)
                self.display_html(_text, self.text_color, "p")
                yDivPoint= data_frame[data_frame[y_value] <= np.percentile(data_frame[y_value],percentile)].max()[y_value]
            if not top_x == None:
                #Ascending means smallest to largest,
                _text = "Top x =" + str(data_frame.sort_values(y_value,ascending = True).head(top_x).shape[0])
                self.display_html(_text, self.text_color, "p")
                yDivPoint = data_frame.sort_values(y_value,ascending = True).head(top_x).max()[y_value]
        
        yMin = round(yMin, 3)
        yDivPoint = round(yDivPoint, 3)
        yMax = round(yMax, 3) 
        
        def get_next_prev_fraction(num):#local method
            n = num
            if isinstance(n, float):
                s = str(n)
                _len = s[::-1].find('.')
                add = "0."
                if (_len > 0):
                    for x in range(_len-1):
                        add = add + "0"
                    add = add +"1"
                _next = round (n + float(add), _len)
                _prev = round (n - float(add), _len)
            return _next, _prev

        _, yDivPoint = get_next_prev_fraction(yDivPoint)
        bins = [yMin-1, yDivPoint, yMax+1]#adding + and -1 to account for any decimal issues
                          
        impact_col_name = "impact"    

        data_frame[impact_col_name] = pd.cut(data_frame[y_value], bins, labels=outcome)
              
        x = "* Output '" + str(y_value) + "' grouped in range:"
        text = text + "A high output is defined as a relevance(score) between " + str(yDivPoint) + " and " + str(yMax)+"<br>"
        text = text + "A low output is defined as a relevance(score) between " + str(yMin) + " and " + str(yDivPoint)
        self.display_html(text, self.text_color, "p")

        sequential_color_list = [
            px.colors.sequential.Greens, 
            px.colors.sequential.Greys]

        
        fig2 = self.plot_donut( list((attributes_list)), 
                                 data_frame[(data_frame[impact_col_name]=="Pos")] ,
                                 w=500, h=500, 
                                 y_high_positive = True, 
                                 title = "Positive impact across groups", 
                                 show_individual = False) 
        
        fig3 = self.plot_donut( list((attributes_list)), 
                                 data_frame[(data_frame[impact_col_name]=="Neg")] ,
                                 w=500, h=500, 
                                 y_high_positive = True, 
                                 title = "Negative impact across groups", 
                                 show_individual = False) 
        
        df_ratios = data_frame[attributes_list+[impact_col_name]+[y_value]].groupby(attributes_list+[impact_col_name]).count().reset_index().rename(columns={y_value: "values"})     
        
        _total =  df_ratios['values'].sum()
        _total_pos =  df_ratios[(df_ratios[impact_col_name]=="Pos")]['values'].sum()
        _total_neg =  df_ratios[(df_ratios[impact_col_name]=="Neg")]['values'].sum()
        
        _text = "Total Number of samples: " + str( _total)
        _text = _text + " (Positive: " + str(_total_pos) + ", Negative: "+ str(_total_neg) + ")"
        self.display_html(_text, self.text_color, "p")
        num_unique_out = len(data_frame[y_value].unique())
        _text = "*Note the number may vary from the top-n specified due to multiple records having same value, the number of unique outputs is " + str(num_unique_out)
        self.display_html(_text, self.text_color, "p")
        one, isto = self.ratio(_total_pos, _total_neg)
        df_ratios['labels'] =  df_ratios.apply(lambda row : self.create_label(row[attributes_list]), axis = 1) 
        df_ratios['labels'].fillna(0,inplace=True)
        attr_list = []
        attr_list.append("All")
        isto_list = []
        isto_list.append(isto)
        pcnt_list = []
        pcnt_list.append((_total_pos/(_total_pos+_total_neg))*100)
        for label in df_ratios['labels'].unique():
            pos = df_ratios[(df_ratios['labels']==label) & (df_ratios[impact_col_name]=="Pos")]['values']
            neg = df_ratios[(df_ratios['labels']==label) & (df_ratios[impact_col_name]=="Neg")]['values']
            one, isto = self.ratio(pos.values[0], neg.values[0])
            attr_list.append(label)
            isto_list.append(isto)
            pcnt = (pos.values[0]/(pos.values[0]+neg.values[0]))*100
            if math.isnan(pcnt):
                pcnt_list.append(0)
            else:
                pcnt_list.append(pcnt)
        
        fig4 = go.Figure()
        fig4.add_trace(go.Bar(
                            x=attr_list,
                            y=isto_list,
                            marker_color='indianred'
                        ))


            # Here we modify the tickangle of the xaxis, resulting in rotated labels.
        fig4.update_layout(xaxis_tickangle=-45, 
                           title = "Ratio of Positive to Negative (for each 1 positive effect)",
                          autosize=False,
                          width=900,
                          height=400,)
        
        fw4 = go.FigureWidget(fig4)
        fig5 = go.Figure()
        fig5.add_trace(go.Bar(
                            x=attr_list,
                            y=pcnt_list,
                            marker_color='indianred'
                        ))


            # Here we modify the tickangle of the xaxis, resulting in rotated labels.
        fig5.update_layout(xaxis_tickangle=-45, 
                           title = "Percentage of positive outcomes per group",
                          autosize=False,
                          width=900,
                          height=400,)
        
        fw5 = go.FigureWidget(fig5)
        
        hbox1 = widgets.HBox([fig2, fig3])
        hbox2 = widgets.HBox([fw4])
        hbox3 = widgets.HBox([fw5])
        display(widgets.VBox([hbox1, hbox2, hbox3]))

        del fig2
        del fig3
        del fig4
        del fig5
        
        num_of_attributes = len(attributes_list)        
        def powerset(iterable):
            "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
            s = list(iterable)
            res = chain.from_iterable(combinations(s, r) for r in range(len(s)+1))
            list_set = list(res)
            final_list = []
            for val in list_set:
                if len(val) == 2:
                    final_list.append(val)
            return final_list
        fig = go.Figure()
        if num_of_attributes > 1:
            
            comb = powerset(attributes_list)
            #for each combination of 2 protected attributes 
            for a, pos in zip (comb, range(len(comb))):
                df = data_frame[[a[0]]+[a[1]]+[impact_col_name]+[y_value]].groupby([a[0]]+[a[1]]+[impact_col_name]).count().reset_index().rename(columns={y_value: "values"})
                _total =  df['values'].sum()
                _total_pos =  df[(df[impact_col_name]=="Pos")]['values'].sum()
                _total_neg =  df[(df[impact_col_name]=="Neg")]['values'].sum()
                color_index = 7
                color_index_g = 7
                for value in df[a[1]].unique():
                    green = px.colors.sequential.Greens[color_index]
                    if color_index != 0:
                        color_index = color_index - 1
                    else:
                        color_index = 7
                    
                    name = a[1]+"_"+str(value)
                    
                    temp = df[(df[a[1]]==value) & (df[impact_col_name]=="Pos")]
                    _pcnt = list (  round((temp["values"]/_total) * 100 , 2)) 
                    _pcnt_from_total_pos = list ( round(  (temp["values"]/_total_pos) * 100, 2)) 
                    
                    for x in range (len(_pcnt_from_total_pos)):
                        _pcnt_from_total_pos[x] = str(_pcnt_from_total_pos[x]) + "% of all Positive"
                    fig.add_trace(go.Bar(
                            y=df[a[0]].unique(),
                            x=temp["values"],
                            name='Positive-' + name,
                            text = _pcnt_from_total_pos,
                            textposition='auto',
                            orientation='h',
                            marker=dict(
                            color=green,
                            line=dict(color='rgba(58, 71, 80, 1.0)', width=2)
                            )
                        ))
                for value in df[a[1]].unique():
                    grey = px.colors.sequential.Oranges[color_index_g]
                    if color_index_g != 0:
                        color_index_g = color_index_g - 1
                    else:
                        color_index_g = 7
               
                    name = a[1]+"_"+str(value)
                    temp = df[(df[a[1]]==value) & (df[impact_col_name]=="Neg")]
                    _pcnt = list ( round(  (temp["values"]/_total) * 100 , 2) )
                    _pcnt_from_total_neg = list ( round(  (temp["values"]/_total_neg) * 100, 2) )
                    for x in range (len(_pcnt_from_total_neg)):
                        _pcnt_from_total_neg[x] = str(_pcnt_from_total_neg[x]) + "% of all Negative"
                    fig.add_trace(go.Bar(
                            y=df[a[0]].unique(),
                            x=temp["values"],
                            name='Negative_' + name,
                            text = _pcnt_from_total_neg,
                            textposition='auto',
                            orientation='h',
                            marker=dict(
                            color=grey,
                            line=dict(color='rgba(58, 71, 80, 1.0)', width=2)
                        )
                    ))
                        

            fig.update_layout(title = "Impact of results on: "+ a[0]+ " and "+ a[1], 
                              barmode='stack',
                              xaxis_tickangle=-45,
                              autosize=False,
                              width=900,
                              height=h,
                              legend_orientation="h",
                              margin=dict(l=50,r=50,b=100, t=100,pad=4),
                              xaxis=dict(title=a[1]),
                              yaxis=dict(title=a[0]), 
                             )
            fig.show()
            del fig
            
        else:#only one protected group so just show it   
            df = data_frame[attributes_list+[impact_col_name]+[y_value]].groupby(attributes_list+[impact_col_name]).count().reset_index().rename(columns={y_value: "values"})
            temp = df[(df[impact_col_name]=="Pos")]
            _total =  df['values'].sum()
            _total_pos =  df[(df[impact_col_name]=="Pos")]['values'].sum()
            _total_neg =  df[(df[impact_col_name]=="Neg")]['values'].sum()
            fig.add_trace(go.Bar(
                            y=df[attributes_list[0]].unique(),
                            x=temp["values"],
                            name='Positive',
                            text = "add %",
                            textposition='auto',
                            orientation='h',
                            marker=dict(
                            color="green",
                            line=dict(color='rgba(58, 71, 80, 1.0)', width=2)
                        )
                    ))
            
            temp = df[(df[impact_col_name]=="Neg")]
            fig.add_trace(go.Bar(
                            y=df[attributes_list[0]].unique(),
                            x=temp["values"],
                            name='Negative',
                            text = "add %",
                            textposition='auto',
                            orientation='h',
                            marker=dict(
                            color="orange",
                            line=dict(color='rgba(58, 71, 80, 1.0)', width=2)
                        )
                    ))
            
            fig.update_layout(title = "Impact of results on: "+ str(attributes_list[0]), 
                              barmode='stack',
                              xaxis_tickangle=-45,
                              autosize=False,
                              width=900,
                              height=h,
                              legend_orientation="h",
                              margin=dict(l=50,r=50,b=100, t=100,pad=4),
#                               xaxis=dict(title=a[1]),
#                               yaxis=dict(title=a[0]), 
                              )
            fig.show()
            del fig
            
        #as we added a column to this dataframe we will remove it here, as maybe a copy was not sent in
        if persist_impact_col == False:
            data_frame.drop([impact_col_name], axis=1, inplace = True)
        else:
            data_frame[impact_col_name].replace(['Pos', 'Neg'], [1, 0], inplace=True)
            data_frame["Transformed_"+y_value] = data_frame[impact_col_name]
            data_frame.drop([impact_col_name], axis=1, inplace = True)
        gc.collect()

    
    #################################################################################################
    #  
    # 
    #################################################################################################   
    def scale_normalise(self, attributes_list, data_frame, w=20, h=20, apply = False, s_type = "MIN_MAX_SCALAR"):
        
        text = "You selected " + str (s_type)
        self.display_html(text, self.text_color, "p")
        scaler = preprocessing.StandardScaler()
        
        data_frame_to_scale = data_frame[attributes_list]
        
        if s_type == "STANDARD_SCALAR":
            scaler = preprocessing.StandardScaler()    
        elif s_type == "MIN_MAX_SCALAR":
            scaler = preprocessing.MinMaxScaler()
        elif s_type == "ROBUST_SCALAR":
            scaler = preprocessing.RobustScaler()
        elif s_type == "NORMALIZER":
            scaler = preprocessing.Normalizer()
      
        
        fig1, (ax1, ax2) = plt.subplots(nrows=2, figsize=(w,h));

        plt.tight_layout()
        scaled_df = scaler.fit_transform(data_frame_to_scale)
        scaled_df = pd.DataFrame(scaled_df, columns=attributes_list)
        
        ax1.set_title('Before Scaling')
        for attribute in attributes_list:
            sns.kdeplot(data_frame[attribute], ax=ax1);
        
       
        ax2.set_title('After '+ s_type)
        for attribute in attributes_list:
            sns.kdeplot(scaled_df[attribute], ax=ax2);
        
        plt.show()
        plt.close(fig1)
        plt.clf()
        
        if apply == True:
            for attribute in attributes_list:
                data_frame[attribute] = scaled_df[attribute]

            return data_frame

        return None
    
    #################################################################################################
    #  
    # 
    ################################################################################################# 
    def label_encoding(self, attributes_list, data_frame):
        # creating initial dataframe
        labelencoder = LabelEncoder()
        return_dict = {}
        for attribute in attributes_list:
            categories = data_frame[attribute].unique()
            temp_df = pd.DataFrame(categories, columns=[attribute])
            # Assigning numerical values and storing in another column
            temp_df[attribute+"_benc"] = temp_df[attribute]
            temp_df[attribute] = labelencoder.fit_transform(temp_df[attribute])
            # Convert this Temp_df into a dictionary
            temp_df.set_index(attribute+"_benc", inplace=True)
            return_dict.update(temp_df.to_dict())
            data_frame[attribute+"_benc"] = data_frame[attribute]
            data_frame[attribute] = labelencoder.fit_transform(data_frame[attribute])
        return return_dict
    
    
    #################################################################################################
    #  
    # 
    #################################################################################################   
    def gcd(self, p, q): 
        if (q == 0): 
            return p
        else:
            return min(q, p)

    def ratio(self, a,b):
        _gcd = self.gcd(a,b)
        one = round(a/_gcd, 2)
        isto = round(b/_gcd, 2)
        if one != 1:
            isto = round (1/one, 2)
            one = 1.0
        return one, isto
    
    
    
    
    #################################################################################################
    #  VIEW FAIRNESS MATRICS AEQUITAS
    # 
    #################################################################################################
    """Difference in means: The difference between the probability for a member of group-a be selected and 
        the probability for a member of group-b to be selected.

        Disparate Impact: the Probability of a member of group-a be selected to be selected divided by
        the probability of a member of group-b to be selected

        False positive rate Ratio of false positive ratio's among protected groups

       False negative rate: Ratio of false negative ratio's among protected groups"""
            
    def view_aequitas_fairness_metrics(self, 
                                       y_column_name,
                                       protected_attributes_list,
                                       X_data_frame, 
                                       y_target, 
                                       y_pred,
                                       _w=600, _h=600, 
                                       y_high_positive = True,
                                        ):

        HIGH_RANGE_POSITIVE = y_high_positive
        aeq_Plot = Plot()
        aeq_Group = Group()
        aeq_Bias = Bias()
        aeq_Fairness = Fairness()

        out1 = widgets.Output(layout={})
        out2 = widgets.Output(layout={})
        out3 = widgets.Output(layout={})
        out4 = widgets.Output(layout={})
        out5 = widgets.Output(layout={})
        out6 = widgets.Output(layout={})
        tab_contents = [out1, out2, out3, out4, out5, out6]
        
        children = tab_contents
        tab = widgets.Tab(style={'description_layout':'auto', 'title_layout':'auto'})
        tab.children = children
        
        tab.set_title(0, "Confusion Matrix")
        tab.set_title(1, "False Positive")
        tab.set_title(2, "False Negative")
        tab.set_title(3, "All Metrics")
        tab.set_title(4, "Disparate Impact")
        tab.set_title(5, "Fairness")
        
        local_layout = {'width': 'auto', 'visibility':'visible'}
        local_layout_hidden  = {'width': 'auto', 'visibility':'hidden'}
        local_style = {'description_width':'initial'}

        _choose_a = widgets.Dropdown(description = "Select protected feature", 
                                     options = protected_attributes_list,
                                     layout = local_layout,
                                     style = local_style)
                
        _choose_b = widgets.Dropdown(description = "Select protected group", 
                                             options = X_data_frame[_choose_a.value].unique(),
                                             layout = local_layout,
                                             style = local_style)
        
        _choose_measure = widgets.Dropdown(description = "Select Metric", 
                                             options = {'False Omission Rate' : 'for',
                                                        'False Discovery Rate' :'fdr',
                                                        'False Positive Rate': 'fpr',
                                                        'False Negative Rate': 'fnr',
                                                        'Negative Predictive Value': 'npv',
                                                        'Precision': 'precision',
                                                        'Predicted Positive Ratio_k' :'ppr',
                                                        'Predicted Positive Ratio_g': 'pprev',
                                                        'Group Prevalence':'prev'},
                                             layout = local_layout,
                                             value = 'precision',
                                             style = local_style)
    
        
        _choose_disparity_measure = widgets.Dropdown(description = "Select Metric", 
                                             options = {'False Positive Rate disparity': 'fpr_disparity',
                                                        'False Negative Rate disparity': 'fnr_disparity',
                                                        'Predicted Positive Ratio_k' : 'ppr_disparity',
                                                        'Predicted Positive Ratio_g disparity' :'pprev_disparity',
                                                        'Precision Disparity': 'precision_disparity',
                                                        'False Discovery Rate disparity': 'fdr_disparity',
                                                        'False Omission Rate disparity': 'for_disparity',
                                                        'True Positive Rate disparity': 'tpr_disparity',
                                                        'True Negative Rate disparity': 'tnr_disparity',
                                                        'npv_disparity': 'npv_disparity',},
                                             layout = local_layout,
                                             value = 'fpr_disparity',
                                             style = local_style)
        
        html = '''<h3>Aequitas: </h3> is an open source bias audit toolkit for machine learning developers, 
        analysts, and  policymakers to audit machine learning models for discrimination and bias,
        and make informed and equitable decisions around developing and deploying predictive risk-assessment 
        tools.'''
        display (HTML(html))
        display(tab)
        
        df_aequitas = pd.concat([X_data_frame[protected_attributes_list],
                                        y_target,
                                        pd.DataFrame(y_pred, 
                                        index=X_data_frame.index)],
                                        axis=1, sort=False);
        
        df_aequitas.rename(columns={y_column_name: 'label_value', 0: 'score'}, inplace=True);
        df_aequitas[df_aequitas.columns.difference(['label_value', 'score'])] = df_aequitas[
                                                    df_aequitas.columns.difference(['label_value', 'score'])].astype(str);
                
        cross_tab, _ =  aeq_Group.get_crosstabs(df_aequitas)
        absolute_metrics = aeq_Group.list_absolute_metrics(cross_tab) 
        #columns not in absolute Matrix
        counts_metrics = list (cross_tab[[col for col in cross_tab.columns if col not in absolute_metrics]].columns.values)
        counts_metrics.remove('model_id') 
        counts_metrics.remove('score_threshold') 
        counts_metrics.remove('k') 
        
        ## Read images from file (because this is binary, maybe you can find how to use ByteIO) but this is more easy
        img2 = open('count.png', 'rb').read()
        img1 = open('absolute.png', 'rb').read()
        ## Create image widgets. You can use layout of ipywidgets only with widgets.
        ## Set image variable, image format and dimension.
        wi1 = widgets.Image(value=img1, format='png', width=300, height=400)
        wi2 = widgets.Image(value=img2, format='png', width=300, height=400)
        ## Side by side thanks to HBox widgets
        sidebyside = widgets.HBox([wi1, wi2])
        ## Finally, show.
        
        with out1:
            clear_output(wait = False)
            display (HTML ("<b>Absolute Metrics across protected groups</b>"))
            display ( cross_tab[['attribute_name', 'attribute_value'] + absolute_metrics].round(2))
            display (HTML ("<br><b>Group counts across protected groups</b>"))
            display ( cross_tab[counts_metrics])
            display(sidebyside)

            
        with out2: #False Positive
            clear_output(wait = False)
            html = '<b>False Positive Rate</b> - the model predicted the subjects outcome was positive when in fact it was not, in other words an incorrect decision TO recommend!<br>'

            if  y_high_positive == True:
                html = html + '''<b>* </b>You have indicated that a high outcome(ranking) has a positive impact on an individual
                                therefore a high false positive rate will have a <b>positive</b> impact on a group
                                '''
            elif y_high_positive == False:
                html = html + '''<b>* </b>You have indicated that a high outcome(ranking) has a negative impact on an individual
                                therefore a high false positive rate will have a <b>negative<b> impact on a group
                                '''
            display (HTML(html))
            fig1, (ax1) = plt.subplots(nrows=1, figsize=(10 ,5));
            ax1 = aeq_Plot.plot_group_metric(cross_tab,'fpr', ax1);
            plt.tight_layout();
            ax1.set_title('False Positive ratios');
            plt.show();
            plt.close(fig1);
            plt.clf();
        
        
        with out3:#False Negative
            clear_output(wait = False)
            html = '<b>False Negative Rate</b> - the model predicted the subjects outcome was negative when in fact it was not, in other words an incorrect decision TO NOT recommend!<br>'
            if  y_high_positive == True:
                html = html +  '''<b>* </b>You have indicated that a high outcome(ranking) has a positive impact on an individual
                                therefore a high false negative rate will have a <b>negative</b> impact on a group.
                                '''
            elif y_high_positive == False:
                html = html + '''<b>* </b>You have indicated that a high outcome(ranking) has a negative impact on an individual
                                therefore a high false negative rate will have a <b>positive impact</> on a group.
                                '''
            display (HTML(html))
            fig1, (ax1) = plt.subplots(nrows=1, figsize=(10 ,5));
            ax1 = aeq_Plot.plot_group_metric(cross_tab,'fnr', ax1);
            plt.tight_layout();
            ax1.set_title('False Negative ratios');
            plt.show()
            plt.close(fig1);
            plt.clf();
            
        with out4:
            clear_output(wait = False)
            html = '''<b>Select the metric to view</b><br>''' 
            if  y_high_positive == True:
                html = html +  '''<b>* </b>You have indicated that a high outcome(ranking) has a positive impact on an individual
                                '''
            elif y_high_positive == False:
                html = html + '''<b>* </b>You have indicated that a high outcome(ranking) has a negative impact on an individual
                               '''
            display (HTML(html))
            def show_any(choose_measure):
                fig1, (ax1) = plt.subplots(nrows=1);
                ax1 = aeq_Plot.plot_group_metric(cross_tab, choose_measure, ax1)
                plt.tight_layout()
                ax1.set_title(choose_measure)
                plt.show()
                plt.close(fig1)
                plt.clf()
            interact(show_any, choose_measure = _choose_measure);
   
        with out5:
            clear_output(wait = False)
            dict_of_controls = {}
            
            dis_imp_html = '''<b>Disparate Impact:</b>. A decision-making process suffers from disparate impact if the outcome 
            of the decision disproportionately benefits one group or disproportionately hurts another group.
            It generally results from unintentional discrimination in decision-making systems.
            Disparities are calculated as a ratio of a metric for a group of interest compared to a reference group. 
            For example, the False Negative Rate Disparity for Group-A compared to a reference Group-B is: FNR-B/FNR-A
            The calculated disparities are in relation to a reference group, which will always 
            have a disparity of 1.0. Disparate impact is often measured by the eighty percent or four-fifths rule. '''
            
            display (HTML(dis_imp_html))
            
            display (HTML("<b>Select a reference group for each protected feature for comparison:</b>"))
            
            for feature in protected_attributes_list:
                dict_of_controls[feature] = widgets.Dropdown(description = "-" + feature +  "- Ref group", 
                                             options = X_data_frame[feature].unique(),
                                             layout = local_layout,
                                             style = local_style)
            for c in dict_of_controls:
                display(dict_of_controls[c])
                
                
            
            display (HTML("<b>Select the metric for which you want to view disparities:</b>"))
            def show_disparity(choose_disparity_measure, button): #local method within view_protected() funct
                _ref_groups_dict = {}   
                for c in dict_of_controls:
                    _ref_groups_dict[c] = str(dict_of_controls[c].value)
                
                disparity = aeq_Bias.get_disparity_predefined_groups(cross_tab, 
                                                                     original_df=df_aequitas,
                                                                     ref_groups_dict=_ref_groups_dict, 
                                                                     alpha=0.05,
                                                                     mask_significance=True); 
               
                num_rows = math.ceil( (len(protected_attributes_list))/2)
                
                fig = plt.figure(figsize=(12 ,6*num_rows))
                plt.tight_layout()
                ax_dict = {}
                for x, num in zip (protected_attributes_list, range(len(protected_attributes_list))):
                    ax_dict[x] = plt.subplot(1, 2, num+1)
                    ax_dict[x] = aeq_Plot.plot_disparity(disparity, 
                                             group_metric=choose_disparity_measure, 
                                             attribute_name=x,
                                             significance_alpha=0.05,
                                             fig = fig,
                                             ax = ax_dict[x]);
                   
                if  y_high_positive == True:
                    html =   '''<b>* </b>You have indicated that a high outcome(ranking) has a positive impact on an individual
                                therefore a XXX disparity score will have a <b>negative</b> impact on a group.<br>
                                '''
                elif y_high_positive == False:
                    html = '''<b>* </b>You have indicated that a high outcome(ranking) has a negative impact on an individual
                                therefore a therefore a XXX disparity score  will have a <b>positive impact</> on a group.<br>
                                '''
                display(HTML(html))
                
                plt.show()
                display(HTML('''Sized based on group size, color based on disparity magnitude<br>
                                Reference groups are displayed in grey with disparity = 1. <br>
                                Disparities greater than 10x will show as 10x.<br>
                                Disparities less than 0.1x will show as 0.1x.<br>
                                Statistical siginificance(default 0.05) will show as ** on square.'''))
                plt.close(fig1)
                plt.clf()
                pd.set_option('display.max_columns', None)
                display (HTML("<b>All Calculated values:</b>"))
                display (disparity)
                
                with out6:
                    clear_output(wait = False)
                    for ref in _ref_groups_dict:
                        display(HTML("Reference group is " +_ref_groups_dict[ref] + " for " + ref))
                        display(HTML("Green bar indicates Fair.<br>Red bar indicates unfair."))
                    group_val_fairness= aeq_Fairness.get_group_value_fairness(disparity)
                    parity_detrminations = aeq_Fairness.list_parities(group_val_fairness)
                    aeq_Plot.plot_fairness_group_all(group_val_fairness, ncols=5, metrics = "all")
                    display (group_val_fairness[['attribute_name', 'attribute_value']+parity_detrminations])
                
                
            interact(show_disparity, 
                     choose_disparity_measure = _choose_disparity_measure,
                     button = widgets.ToggleButton(
                         description='Apply selected Reference group',
                         layout = local_layout,
                         style = local_style),
                     )

        
                
            di_rule_html =''''Eighty percent rule is a rule of thumb or a guideline applied to the measurement of 
        disparate impact whereby a selection rate for  a protected group-A which is less than 80% of the
        selection rate for group-B, where group-B is the group with the highest selection rate, will generally 
        be considered as evidence of adverse impact against group-A. The eighty percent rule does not
        incorporate probability distribution to determine if the disparity is as a result of chance and therefor is not a definitive 
        test.''' 
    

# Precision or  positive predictive value(PPV) is the proportion of the individuals with a positive test result for which the true
# condition is positive. This rate is sometimes called the precision.
#                     print ('''The ppr and difference in means are more generalized fairness metrics, since they 
#                     only consider how much the outcome differs between the protected groups. 
#                     This plot will show a Gi% probability for a member of group Gi to obtain a positive outcome. 
#                     The rate of this metric denotes Disparate Impact''')
             
        
#                 .plot_fairness_group(group_val_fairness, group_metric='fpr')
#                 .plot_fairness_group(group_val_fairness, group_metric='fnr'
#                 .plot_fairness_group(group_val_fairness, group_metric='ppr')


#                 html_parity = """Considering the rule of thumb a parity should be between 0.5 and 1.5, 
#                 The disparate impact still shows that men are somewhat favored over women,
#                 but the difference is not much reduced. Based on the false negative ratio parity 
#                 we can say that we managed to mitigate the bias, but the model is still unfair. """
                
                
#         except Exception as e:
#             self.display_html("Something went wrong viewing Aequitas fairness metrics", self.text_color, "h4")
#             print (e)
        gc.collect()

    def visualise_RMSE_model_eval(self, y_train, y_test, y_pred_train, y_pred_test):
        y_predictedTrain = y_pred_train
        y_predictedTest = y_pred_test
        y_test = y_test
        y_train = y_train
        '''We will be using Root mean squared error(RMSE) and Coefficient of Determination(R² score) to evaluate our model.
        RMSE is the square root of the average of the sum of the squares of residuals. 
        The RMSE is the square root of the variance of the residuals. 
        #It indicates the absolute fit of the model to the data–how close the observed data points are to the model’s
        #predicted values. Whereas R-squared is a relative measure of fit, RMSE is an absolute measure of fit. 
        #As the square root of a variance, RMSE can be interpreted as the standard deviation of the unexplained variance, 
        #and has the useful property of being in the same units as the response variable. 
        #Lower values of RMSE indicate better fit. 
        #RMSE is a good measure of how accurately the model predicts the response, and it is the most important criterion
        #for fit if the main purpose of the model is prediction.

        #Coefficient of Determination(R² score) - The best possible score is 1.0 and it can be negative (because the model can be arbitrarily worse). 
        #A constant model that always predicts the expected value of y, disregarding the input features, 
        #would get a R^2 score of 0.0. #R-squared is between 0 and 1, Higher values are better because it means 
        #that more variance is explained by the model.'''

 
        display("*****EVALUATING MODEL WITH TRAINING DATA:***")
        rmse = mean_squared_error(y_train, y_predictedTrain)
        r2 = r2_score(y_train, y_predictedTrain)
        display(' Root mean squared error: '+ str(rmse))
        display(' R2 score: '+ str(r2))

        display("*****EVALUATING MODEL WITH TEST DATA:******")
        rmse = mean_squared_error(y_test, y_predictedTest)
        r2 = r2_score(y_test, y_predictedTest)
        display(' Root mean squared error: ' + str(rmse))
        display(' R2 score: '+ str(r2))


        n_train = len(y_train)


        plt.figure(figsize=(20, 10))
        plt.plot(range(n_train), y_train, label="train")
        plt.plot(range(n_train, len(y_test) + n_train), y_test, '-', label="test")
        plt.plot(range(n_train), y_predictedTrain, '--', label="prediction train")

        plt.plot(range(n_train, len(y_test) + n_train), y_predictedTest, '--', label="prediction test")
        plt.legend(loc=(1.01, 0))
        plt.xlabel("Score")
        plt.ylabel("prediction")
        
    def reload_data (self, pickle_path, data_frame_path, print_report = True):
    # Reload the file
        data_summary = dill.load(open(pickle_path, "rb"))
        data_frame = pd.read_csv(data_frame_path)
        display (HTML('''<b>Values saved in data_summary</b> <br>  
        data_summary.renamed <br>
        data_summary.referenceNames<br>
        data_summary.encoding_dict<br>
        data_summary.df_url<br>
        data_summary.y_value<br>
        data_summary.Y_BINARY<br>
        data_summary.Y_CONTINUOUS<br>
        data_summary.HIGH_RANGE_POSITIVE<br>
        data_summary.protected_x<br>
        data_summary.non_protected_x<br>
        data_summary.all_columns_in_x<br>
        data_summary.proxyAttributes<br>
        data_summary.MLAttributes<br>
        data_summary.html'''))
        if print_report == True:
            print("Number of samples in dataset: ", data_frame.shape[0])
            print ("Output to predict: ", data_summary.y_value)
            print ("Output is continuous: ", data_summary.Y_CONTINUOUS)
            print ("Output is binary: ", data_summary.Y_BINARY)
            print ("Num of unique outputs: ", len(data_frame[data_summary.y_value].unique()))
            print ("Min output: ", data_frame[data_summary.y_value].min())
            print ("Max output: ", data_frame[data_summary.y_value].max())
            print ("High output range is positive? ", data_summary.HIGH_RANGE_POSITIVE)
            #print ("High ranking is by default ", data_summary.y_mid_rank, " to ", data_summary.y_max_rank)
            #print ("Low ranking is by default ",data_summary.y_min_rank , " to ", data_summary.y_mid_rank)
            print ("")
            print ("Protected input features", data_summary.protected_x)
            print ("Other input features", data_summary.non_protected_x)
            print("")
            print ("Summary of Data transformation per input feature:")
            for key in data_summary.html:
                if data_summary.html[key] != "":
                    print ("************")
                    print (key)
                    display (HTML (data_summary.html[key]))
        
            print ("")
            print ("Input Features identified as possible proxies to protected Features: ", data_summary.proxyAttributes)
            print ("")
            print ("Input Features identified as possibly output from another ML Model: ", data_summary.MLAttributes)
            print ("")
            print ("")
            print ("The X and y Values to be used in Model Train/Test/Validate")
            display (data_frame[data_summary.protected_x+data_summary.non_protected_x+[data_summary.y_value]].head())
            print ("")
            print ("")
            print ("")
            print ("All versions of all columns to be used for analysis of Test/Validate")
        return data_frame, data_summary
    
    
    def get_train_test_split_data(self, data_frame, data_summary, include_protected):
        if include_protected == True:
            cols = data_summary.protected_x + data_summary.non_protected_x
        else:
            cols = data_summary.non_protected_x
        return  data_frame.drop(data_summary.y_value, axis = 1), data_frame[data_summary.y_value], cols
    
    
    
    def shap_analysis(self, shap_values, explainer, x, data_summary):
        try:
            shap.initjs()
        except:
            print ( 'shap.initjs() failed to load javascript')
        
        outOverview = widgets.Output(layout={})
        out1 = widgets.Output(layout={})
        out2 = widgets.Output(layout={})
        out3 = widgets.Output(layout={})
        out4 = widgets.Output(layout={})
        out5 = widgets.Output(layout={})
        tab_contents = [out1, out2, out3, out4, out5]
        
        children = tab_contents
        tab = widgets.Tab(style={'description_layout':'auto', 'title_layout':'auto'})
        tab.children = children
        
        tab.set_title(0, "Summary Importance plot")
        tab.set_title(1, "Importance plot")
        tab.set_title(2, "Dependence plot")
        tab.set_title(3, "Individual force plot")
        tab.set_title(4, "Collective force plot")
        
        local_layout = {'width': 'auto', 'visibility':'visible'}
        local_layout_hidden  = {'width': 'auto', 'visibility':'hidden'}
        local_style = {'description_width':'initial'}
        display(outOverview)
        display(tab)
        
        
        _choose = widgets.Dropdown(description = "Select Feature", 
                                    options = data_summary.protected_x + data_summary.non_protected_x,
                                    layout = local_layout,
                                    style = local_style)
        all_comb = {}
        for f in data_summary.protected_x:
            for a in  x[f].unique():
                all_comb[(f+":"+ str(a))] = a
        
        _protected = widgets.Dropdown(description = "Filter by Protected Feature", 
                                     options = all_comb,
                                     layout = local_layout,
                                     style = local_style)
        
        toggle = widgets.ToggleButton(
                            value=False,
                            description='Generate',
                            disabled=False,
                            button_style='', # 'success', 'info', 'warning', 'danger' or ''
                            
                            )
        
        with outOverview:
            display (HTML('''<h3>SHAP</h3>(SHapley Additive exPlanations) KernelExplainer is a 
            model-agnostic method which builds a weighted linear regression by using training/test data, 
            training/test predictions, and whatever function that predicts the predicted values. 
            SHAP values represent a feature's responsibility for a change in the model output.
            It computes the variable importance values based on the Shapley values from game theory, 
            and the coefficients from a local linear regression. </br>
            see: https://papers.nips.cc/paper/7062-a-unified-approach-to-interpreting-model-predictions.pdf <br>

            It offer a high level of interpretability for a model, through two distinct approaches:

            <b>Global interpretability</b> — the SHAP values can show how much each predictor contributes, 
            either positively or negatively, to the target variable. Similar to a variable importance plot however it also indicates the positive or negative relationship between each feature and the target output.

            <b>Local interpretability</b> — each observation is assigned it's own SHAP value. 
            This provides a very granular level of transparency and interpretability where we can 
            determine why an individual cases receive a specific prediction  and the contribution of 
            each feature to the prediction. Generally speaking variable importance algorithms usually 
            only show the results across the entire dataset but not on each individual case.'''))
            
        with out1:
          
            html_desc = '''
            <b>Summary importance plot </b><br><b>Feature importance:</b> Variables are ranked in descending order. The top variables contribute more to the model than the bottom ones and thus have high predictive power.<br>
            ''' 
            wi1 = widgets.Output(layout=Layout(width='60%'))
            with wi1:
                shap.summary_plot(shap_values, x, plot_type="bar"); 
            wi2 = widgets.HTML(value=html_desc,layout=Layout(width='30%') ) ; 
            sidebyside = widgets.HBox([wi1, wi2])
            display (sidebyside)
        with out2:
            display (HTML('''<b>Importance plot:</b> lists the most significant variables in descending order. 
                          The top variables contribute more to the model than the bottom ones and thus have high predictive power.'''))
            html_desc = '''
            <b>Feature importance:</b> Variables are ranked in descending order.<br>
            <b>Impact:</b> Horizontal location indicates if effect of feature is associated with a higher or lower prediction.<br>
            <b>Original value:</b> Colour indicates if feature variable is high(red) or low(blue) for the particular observation.<br>
            <b>Correlation:</b> A high or low impact(indicated by colour), a positive or negative impact(indicated by position on x-axis)
                '''  
            wi1 = widgets.Output(layout={})
            with wi1:
                shap.summary_plot(shap_values, x);
            wi2 = widgets.HTML(value=html_desc,layout=Layout(width='30%') )  
            sidebyside = widgets.HBox([wi1, wi2])
            display (sidebyside)
                
        with out3:
            
            display (HTML ("The decision variable is " + str(data_summary.y_value)))
            display (HTML ('''To understand how a single feature effects the output of the model a 
                             dependency plot plots the SHAP value of  that feature vs. the value of 
                             the feature for all the examples in a dataset.'''))
            def show_dependancy_plot(choose):
                html_desc = '''The dependency plots show relationship between the target ('''+ data_summary.y_value +  ''') 
                   and the selected feature ('''+ choose + ''') to review if it is linear, monotonic or 
                  more complex.  The additionla variable is the variable that the selected feature (''' + choose + ''') 
                  interacts with the most frequently. Vertical dispersion at a single value represents interaction 
                  effects with the other features.  '''
                display (HTML(html_desc))
                display (shap.dependence_plot(choose, shap_values, x))
            interact(show_dependancy_plot, choose = _choose);
    
        with out4:
            display (HTML ("The decision variable is " + str(data_summary.y_value)))
            display (HTML ('''<b>Individual Force plot</b> shows the features which each contribute to push the model output 
                            from the base value (the average model output over the dataset passed) to the
                            model output. Features pushing the prediction higher are shown in red, 
                            those pushing the prediction lower are shown in blue.'''))
            # visualize the first prediction's explanation (use matplotlib=True to avoid Javascript)
            display (HTML ("<b>Generate random sample to investigate:</b>"))
            def show_individual_force_plot(protected, toggle):
                feat = _protected.label.split(':')[0]
                index = x[x[feat] == _protected.value].sample(1).index[0]
                display (shap.force_plot(explainer.expected_value, 
                                     shap_values[index,:], 
                                     x.iloc[index,:],
                                     matplotlib=True))
            
            interact(show_individual_force_plot, protected = _protected, toggle = toggle);
         
        with out5:
            display (HTML ("The decision variable is " + str(data_summary.y_value)))
            display (HTML ('''<b>Collective Force plot</b> A combination of all individual force plots, each rotated 90 degrees, and stacked
                                horizontally, to explanation an entire dataset.'''))
            display (shap.force_plot(explainer.expected_value, 
                                     shap_values, 
                                     x))

    def get_protected (self, summary):
        return summary.protected_x
    
    def get_protected_before_merge (self, summary):
        return summary.all_columns_in_x
        print (list([all_columns_in_x.contains('_bm')]))
        print (list(X_train.columns[X_train.columns.str.contains('_benc')]))


    def get_protected_before_transform (self, summary):
        all_cols = summary.all_columns_in_x
        prot = summary.protected_x

        new_prot = []
        for f in prot:
            found = False
            if f+'_bm' in all_cols:
                new_prot.append(f+'_bm')
                found = True
            if f+'_benc' in all_cols:
                new_prot.append(f+'_benc')
                found = True
            if f.endswith('_oh_benc'):
                new_prot.append(f)
                found = True
        
            if found == False:
                new_prot.append(f)
        return new_prot
    
    def prepare_to_shap(self, X_in, count = 100, save_to_path = './'):
        x = shap.sample( X_in, count)
        x = x.reset_index(drop=True)
        explainer = shap.KernelExplainer(logistic_reg_model.predict, x ) # The second argument is the "background" dataset; a size of 100 rows is gently encouraged by the code
        shap_values = explainer.shap_values(x, l1_reg="num_features(10)")
        print(f'length of SHAP values: {len(shap_values)}')
        print(f'Shape of each element: {shap_values[0].shape}')
        path =  save_to_path + "shap_values.pickle"
        print ("Shap_values saved to", path)
        dill.dump(shap_values, file = open(path, "wb"))
        path =  save_to_path + "shap_explainer.pickle"
        print ("Shap_explainer saved to", path)
        dill.dump(explainer, file = open(path, "wb"))
        path =  save_to_path +"shap_x.pickle"
        print ("Shap_explainer saved to", path)
        dill.dump(x, file = open(path, "wb"))
        return explainer, shap_values, x

    def reload_shap_data (self, _path):
        path = _path 
        shap_values_path = path + "/shap_values.pickle"
        explainer_path = path + "/shap_explainer.pickle"
        x_path = path + "/shap_x.pickle"
        shap_values = dill.load(open(shap_values_path, "rb"))
        explainer = dill.load(open(explainer_path, "rb"))
        x = dill.load(open(x_path, "rb"))
        # Reload the file
        return shap_values, explainer, x